# TP — Réseaux convolutifs

Dans ce TP, nous allons examiner plusieurs manières de faire de la classification avec des réseaux de neurones. Nous allons d'abord commencer par un réseau de neurones simple, complètement connecté. Puis, nous introduirons les réseaux convolutifs. Les tâches sur lesquelles nous travaillerons sont des tâches de classification, on cherche à *prédire* à quelle classe connue appartient une image, à partir d'un modèle qui se sera *entraîné* sur des données *étiquetées*.

**Consignes :**
Le TP est noté et je vous demande un rendu individuel, en indiquant cependant le nom de la personne avec qui vous avez travaillé durant la séance.
La remise des travaux s'effectue sur Moodle, au format ipynb ou pdf ([**cliquez ici**](https://moodle-sciences-24.sorbonne-universite.fr/mod/assign/view.php?id=117975)). La date limite est le 4 décembre 2024 à 18h (heure de Paris).

## Quelques imports généraux

In [ ]:
pip install tensorflow #j'install trensorflow directement avec vscode

In [5]:
from __future__ import print_function
from tensorflow.keras.utils import to_categorical
import numpy as np
np.random.seed(1671)

: 

: 

: 

## Les données MNIST

Nous allons dans un premier temps travailler avec la [base bien connue des chiffres MNIST](https://yann.lecun.com/exdb/mnist/). L'intérêt de cette base est notamment d'être intégrée directement dans les bibliothèques de code, ce qui permet de travailler facilement dessus, et de comparer les performances de nos modèles et algorithmes avec d'autres.

On définit le paramètre `NB_CLASSES` : en sortie, il y aura 10 chiffres, nous allons demander une sortie (prédiction) dans l'une des 10 classes (0-9)

In [ ]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

RESHAPED = 784
NB_CLASSES = 10  

X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = to_categorical(y_train, NB_CLASSES)
Y_test = to_categorical(y_test, NB_CLASSES)

**Question :** Expliquez les traitements faits pour chaque image. Comment considère-t-on les images ?

**Question :** Regardez et commentez le format de `Y_train` et `Y_test`.

## Un premier réseau, avec Keras

Nous allons, dans cette première partie, travailler sur un premier réseau de neurones **qui n'utilisera pas les convolutions**. Le but sera d'apprendre à utiliser des rudiments de la bibliothèque Keras, et d'avoir une première *baseline* de performance. Une *baseline* est une expérience de référence, en général avec un modèle peu complexe, qui permet de percevoir le gain obtenu avec le modèle que l'on souhaite évaluer.

La [bibliothèque Keras](https://keras.io/) permet d'interagir en Python avec les algorithmes d'apprentissage les plus connus, notamment Tensorflow, PyTorch. Ces dernières années, Keras a été absorbée par TensorFlow. Keras permet une expérimentation rapide avec les réseaux de neurones, se concentrant sur des aspects haut-niveau.

### Quelques imports spécifiques à Keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

### Quelques paramètres :

- VERBOSE : nous souhaitons que l'algorithme nous donne des détails
  textuels sur son exécution
- NB_EPOCH : nous allons itérer 200 fois pour l'apprentissage de notre réseau
- BATCH_SIZE : les images seront regroupées dans des lots d'une
  certaine taille
- OPTIMIZER : il y a diverses techniques pour paramétrer
  l'apprentissage. SGD est la "stochastic gradient descent".
- VALIDATION_SPLIT : la fraction de données utilisées pour valider notre
  entraînement de modèle

Une *epoch* désigne le traitement de toutes les données du dataset.
Pour paralléliser (et accélérer) les calculs, les données du dataset sont regroupés en *lots* plus petits, de taille `BATCH_SIZE`. Le nombre d'*epoch* `NB_EPOCH` et la taille des lots `BATCH_SIZE` sont des hyper-paramètres de l'apprentissage : il faut les définir (et les ajuster) pour réaliser un apprentissage efficace.

Pendant une *epoch*, il y a des itérations, chacune traitant un lot. Par exemple, si une itération traite 10 images d’un ensemble de 1000 images avec une taille de batch de 10, il faudra 100 itérations pour terminer une époque.

In [ ]:
NB_EPOCH = 200
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = SGD()
VALIDATION_SPLIT=0.2

### Le modèle

Keras propose plusieurs façons différentes de définir un réseau de neurones. La façon la plus courante pour les réseaux à propagation avant (feedforward), qui empilent les couches de façon séquentielle, est d’utiliser la classe `Sequential()`.

In [ ]:
model = Sequential()

`model` contient représente ainsi un réseau de neurones vide (pour l’instant). Il est possible d’ajouter des couches à l’aide de la méthode add. De nombreuses couches sont prédéfinies dans Keras, comme les couches entièrement connectées (couches linéaires dites Dense) ou les fonctions d’activation standard.

Nous allons seulement ajouter une couche Dense, et une activation. Donc pas de couche cachée, dans un premier temps :

In [ ]:
model.add(Dense(NB_CLASSES, input_shape=(RESHAPED,)))
model.add(Activation('softmax'))

Keras permet d'afficher un résumé du modèle :

In [ ]:
model.summary()

**Question :** À quoi correspond le nombre de paramètres affiché ?

En plus de la définition de l’architecture, nous avons encore besoin de spécifier deux éléments à Keras avant d’entraîner notre modèle : une fonction de coût (*loss*) et une méthode d’optimisation. Ces paramètres sont spécifiés lors de la phase de compilation du modèle à l’aide de la méthode `.compile()`. Nous allons utiliser l’entropie croisée (`categorical_crossentropy`) comme fonction de coût et la descente de gradient stochastique (*stochastic gradient descent* ou sgd).

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=OPTIMIZER,
              metrics=['accuracy'])

### L'entraînement

Les méthodes `fit()` et `evaluate()` sont essentielles :
- `fit()` lance l'apprentissage sur les données
- `evaluate()` permet de calculer automatiquement les performances selon l'indicateur choisi (`metrics`)

In [ ]:
history = model.fit(X_train, Y_train,
                    batch_size=BATCH_SIZE, epochs=NB_EPOCH,
                    verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

Une fois le modèle entraîné, nous pouvons l'évaluer sur l'ensemble de test qui contient de nouveaux exemples inédits. De cette manière, nous obtenons la valeur minimale atteinte par la fonction objective et la meilleure valeur atteinte par la métrique d'évaluation. 

L'ensemble d'apprentissage et l'ensemble de test sont, bien entendu, rigoureusement séparés. Il est inutile d'évaluer un modèle sur un exemple qui a déjà été utilisé pour l'apprentissage. L'apprentissage est essentiellement un processus destiné à généraliser des observations inédites et non à mémoriser ce qui est déjà connu.

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

**Question :** Commenter les résultats.

### Effet de l'ajout de couches

Pour modifier le modèle, il vous suffit de reprendre le code quelques lignes plus haut, pour redéfinir `model` et mettre les couches que vous voulez.
Dans un premier temps, ajoutez une couche `Dense` dite "cachée", car elle n'est pas directement connectée à l'entrée ou à la sortie du réseau. Ajoutez ensuite une couche d'activation `relu`, puis une autre couche cachée, et une autre `relu`.

```python
N_HIDDEN = 128
model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED,)))
model.add(Activation('relu'))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()
```

**Question :** Actualisez le modèle, refaites l'entraînement, et commentez les résultats.

**Exercice :** Refaites ce qui précède avec une seule couche cachée.
Essayez avec plus de deux couches cachées.

**Question :** Commentez vos résultats.

### Expériences autour d'autres paramètres

#### Dropout

En machine-learning, on utilise parfois du "dropout", c'est-à-dire que certaines connexions entre deux couches sont "délaissées" et n'interviennent pas dans le calcul des poids. Le *dropout* permet (souvent) d'éviter le sur-apprentissage et d'améliorer les performances du modèle.

On caractérise le *dropout* par un hyperparamètre, une probabilité de ne pas tenir compte aléatoirement de certaines valeurs.

**Question :** Ajoutez une couche `Dropout` avec une probabilité `0.3` après les deux couches d'activation `relu`, dans le modèle précédent. Refaites l'entraînement et commentez.

**Optionnel :** On peut s'intéresser à différentes valeurs de dropout, vous pouvez essayer des valeurs entre 0.1 et 0.4, si vous voulez.

#### Nombre d'epoch (pas en classe)

**Question :** Refaites l'entraînement en changeant le nombre d'*epoch*, en essayant plusieurs valeurs (exemples : 10, 50, 100, 200). Tracez une courbe des performances obtenues pour un des modèles ci-dessus (ie, l'accuracy en fonction du nombre d'epoch). Vous n'êtes pas obligés de traver la courbe avec `matplotlib`, si vous êtes plus à l'aise avec un tableur (mais dans ce cas, il faudra bien intégrer dans ce notebook votre image obtenu via le tableur).

#### Optimiseur

Il y a plusieurs techniques pour optimiser l'apprentissage, nous avons commencé avec une "simple" descente de gradient stochastique.

**Question :** Essayez de passer de la `SGD` à `Adam()`, et commentez les résultats.

#### Nombre de neurones dans les couches cachées (pas en classe)

**Question :** Essayez des valeurs différentes pour le nombre de neurones dans les couches cachées (`N_HIDDEN`), par exemple 32, 64, 256, 512, 1024. Commentez vos résultats.

#### Taille des lots

**Question :** Comme pour les autres paramètres, expérimentez les performances du modèle avec quelques valeurs de `BATCH_SIZE` différentes de celle du début, et commentez.

#### Initialisation (pas d'expérience ici)

**Question :** À l’aide de la [documentation de Keras](https://keras.io/api/layers/core_layers/dense/), expliquez comment sont initialisés les paramètres du modèle pour les couches entièrement connectées.

### Sauvegarder un modèle

Remarque : il est possible de sauvegarder un modèle sur le disque, et de le récupérer :
```python
model.save('monMLP')

from tensorflow.keras.models import load_model
model = load_model('monMLP')
```

## Réseaux convolutifs (CNN)

Les réseaux convolutifs profonds (Convolutional Neural Networks ou CNN) sont particulièrement adaptés à la reconnaissance d’images, et nous allons expérimenter ce qu'il en est sur la base MNIST, avant de tester sur d'autres images.

Nous allons travailler autour d'une architecture CNN particulière, LeNet. La première version de LeNet a été proposée en 1989 par le français Yann LeCun, quelques années après sa thèse à Sorbonne Université (alors appelée Université Pierre-et-Marie-Curie). Yann LeCun a reçu le prix Turing (l'équivalent du prix Nobel pour l'informatique) en 2018 pour ces travaux. 

Les réseaux convolutifs ont été créés pour travailler efficacement sur des images. Ils reposent sur 3 grandes idées :
- le champ réceptif local
- les poids partagés
- le *pooling*.

Le champ réceptif local consiste à relier une sous-matrice de l'image initiale à un neurone unique de la couche suivante, de façon à analyser *localement* l'image, avec une opération de convolution. Il y a plusieurs façons de parcourir une image, avec des masques recouvrants ou non, et différentes façons de gérer les conditions au bord.

Les poids partagés consiste à partager les poids dans chaque couche sur toute l'image, de façon à apprendre à repérer des caractéristiques *où qu'elles se trouvent dans l'image*.

Le pooling, c'est une façon de réduire la taille des données manipulées, en agrégeant des valeurs locales pour les résumer par un scalaire qui décrit le contenu de cette région.



In [ ]:
# import the necessary packages

from tensorflow import keras # Import Keras from TensorFlow
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense # Imported layers directly from keras.layers
from keras.datasets import mnist
from keras.optimizers import SGD, RMSprop, Adam
import numpy as np

import matplotlib.pyplot as plt

np.random.seed(1671)  # for reproducibility

### Les données

On travaille toujours avec MNIST.

In [ ]:
IMG_ROWS, IMG_COLS = 28, 28 # input image dimensions
NB_CLASSES = 10  # number of outputs = number of digits
# The input shape needs to be changed to reflect NHWC (Number of samples, Height, Width, Channels)
# Since mnist is grayscale it only has 1 channel.
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 1)
print(INPUT_SHAPE)

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, NB_CLASSES)
y_test = to_categorical(y_test, NB_CLASSES)

**Question :** Remarquez qu'il y a eu une modification des données initiales, elles ne sont plus utilisées comme précédemment. Comment sont-elles organisées ? Pourquoi ?

### Le modèle LeNet

Voici l'archietcture du modèle LeNet :
[img](http://raphael.fournier-sniehotta.fr/files/dcrn/images/LeNet5.png)

En fait, nous allons travailler sur une variante de cette architecture, avec des paramètres légèrement différents ici et là. Voici une classe implémentant cette architecture. Vous devriez commencer à être familiers avec l'empilement séquentiel des couches, les unes après les autres.

Il y a des couches de [convolution `Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/) et de [*pooling* `MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/), dont vous trouverez la documentation en cliquant sur les liens.

In [ ]:
class LeNet:
	@staticmethod
	def build(input_shape, classes):
		model = Sequential()

		conv1 = Conv2D(16,kernel_size=(5, 5),activation='relu',input_shape=(28, 28, 1),padding='valid')
		model.add(conv1)
		model.add(Activation("relu"))
		pool1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))
		model.add(pool1)

		conv2 = Conv2D(32,kernel_size=(5, 5),activation='relu',input_shape=(28, 28, 1),padding='valid')
		model.add(conv2)
		model.add(Activation("relu"))
		pool2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))
		model.add(pool2)

		model.add(Flatten())
		model.add(Dense(100,  input_dim=784, name='fc1'))
		model.add(Activation("relu"))

		# a softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		return model

### Utilisons le modèle : apprentissage

In [ ]:
NB_EPOCH = 20
BATCH_SIZE = 100
VERBOSE = 1
OPTIMIZER = Adam()
VALIDATION_SPLIT=0.2

model = LeNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)
print(model.summary())

**Question :** Lisez la définition du modèle dans la classe LeNet ci-dessus, et commentez son architecture (ie, détaillez quelles couches sont utilisées, expliquez les tailles d'entrée-sortie).

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
	metrics=["accuracy"])

history = model.fit(X_train, y_train,
		batch_size=BATCH_SIZE, epochs=NB_EPOCH,
		verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

score = model.evaluate(X_test, y_test, verbose=VERBOSE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

### Évaluation

Affichons 2 courbes importantes :
- l'évolution de la métrique "accuracy" à chaque *epoch*
- l'évolution de la *loss* (l'erreur)

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

**Question :** Commentez ces courbes.

### Importance de la quantité de données d'apprentissage

Pour bien comprendre l'intérêt de **la quantité de données** dans les modèles profonds, modifiez les données sur lesquelles vous apprenez, pour avoir successivement 5900,
3000, 1800, 600, et 300 images (l'ensemble de validation sera toujours *ce qui reste*). L'ensemble de test a toujours 10 000 images.

**Question :** Tracez une courbe avec les performances du modèle LeNet en fonction de la taille de l'ensemble d'apprentissage.

**Question :** Commentez cette courbe. En particulier, mettez cela en perspective avec l'histoire des réseaux convolutifs, mis au point au début des années 1990, mais pleinement utilisés à partir de 2012.

## Reconnaître des chats et des chiens

Nous allons cette fois travailler avec une autre base, la base d'images CIFAR-10, qui contient 60000 images, de dimensions 32x32 (3 canaux RGB), réparties en 10 classes. En particulier, il y a une classe de chats et une classe de chiens.

[img](http://raphael.fournier-sniehotta.fr/files/dcrn/images/cifar10.png)

Notre objectif sera d'être capable de *prédire* la classe d'une nouvelle image, c'est-à-dire de disposer d'un modèle capable de reconnaître un chat ou un chien sur des images qu'il n'a pas vues auparavant.

### Quelques paramètres et imports

In [ ]:
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop

import matplotlib.pyplot as plt

In [ ]:
# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

#constant
BATCH_SIZE = 128
NB_EPOCH = 20
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()

### Les données

Vous pouvez utiliser le code suivant pour afficher quelques images issues des données, avec leurs étiquettes (ie, leurs classes d'appartenance).

In [ ]:
# CIFAR-10 classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck']

# Select a subset to display
num_images = 10  # Number of images to display
indices = np.random.choice(len(X_train), num_images, replace=False)

# Plot the images
plt.figure(figsize=(15, 5))
for i, idx in enumerate(indices):
    img = X_train[idx]
    label = classes[y_train[idx][0]]  # Get the class name
    file_name = f"{label}{i+1}.jpg"  # Create a file-like name
    
    # Display image
    plt.subplot(1, num_images, i + 1)
    plt.imshow(img)
    plt.title(file_name, fontsize=8)
    plt.axis('off')

plt.tight_layout()
plt.show()

Avec le code suivant, vous pouvez afficher *une* image, dont l'indice est précisé.

In [ ]:
import tensorflow as tf

image_index = 0
label = y_train[image_index][0]  # Get the label from y_train
class_name = classes[label]
print(f"Image index: {image_index}")
print(f"Class: {class_name}")

plt.imshow(X_train[image_index])
plt.legend(class_name)
plt.show()

Maintenant, nous préparons les données pour apprendre.

In [ ]:
#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
 
# convert to categorical
Y_train = to_categorical(y_train, NB_CLASSES)
Y_test = to_categorical(y_test, NB_CLASSES) 

# float and normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

### Le modèle

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
  
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))

model.summary()

**Question :** Que pensez-vous de cette architecture ?

### L'entraînement

Attention, cela peut être assez long, environ 2 minutes par epoch. Réduisez éventuellement le nombre d'epoch avant de lancer la cellule suivante.

In [ ]:
# train
model.compile(loss='categorical_crossentropy', optimizer=OPTIM,
	metrics=['accuracy'])
 
history = model.fit(X_train, Y_train, batch_size=BATCH_SIZE,
	epochs=NB_EPOCH, validation_split=VALIDATION_SPLIT, 
	verbose=VERBOSE)
 
print('Testing...')
score = model.evaluate(X_test, Y_test,
                     batch_size=BATCH_SIZE, verbose=VERBOSE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

In [ ]:
#save model
model_json = model.to_json()
open('cifar10_architecture.json', 'w').write(model_json)
model.save_weights('cifar10.weights.h5', overwrite=True)

### L'évaluation

In [ ]:
# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

**Question :** Commentez ces résultats.

### Prédiction à partir d'images de votre choix

On recharge le modèle sauvegardé précédemment (dans le cas d'un TP où les choses sont faites séquentiellement, l'intérêt n'est pas forcément clair, ça le devient si vous avez des interruptions, évidemment).

In [ ]:
import numpy as np
import scipy.misc
from keras.models import model_from_json
from keras.optimizers import SGD

#load model
model_architecture = 'cifar10_architecture.json'
model_weights = 'cifar10.weights.h5'
model = model_from_json(open(model_architecture).read())
model.load_weights(model_weights)

Nous allons tester, qualitativement (c'est-à-dire sur quelques images, sans faire de statistiques), si notre modèle reconnaît des images de chats à partir de photos venant d'autres datasets.

Vous pouvez tester avec, par exemple, les urls suivantes :
- http://raphael.fournier-sniehotta.fr/files/dcrn/images/cats-dogs-val/cats/cat.2024.jpg
- http://raphael.fournier-sniehotta.fr/files/dcrn/images/cats-dogs-val/cats/cat.2082.jpg
- etc. (de 2000 à 2499 inclus)

À la fin de la cellule, vous aurez un affichage de l'image, redimensionnée en 32x32 pixels. Vous pouvez utiliser l'url complète pour afficher l'image en taille réelle.

Vous pouvez aussi trouver l'url d'une image sur le web (par exemple dans une autre catégorie, comme "avion"/"airplane" ou "camion"/"truck"), et tester les prédictions.

In [ ]:
from google.colab import files as FILE
import requests
import os
import numpy as np
from PIL import Image
from keras.models import model_from_json
from keras.optimizers import SGD

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck']

url = "http://raphael.fournier-sniehotta.fr/files/dcrn/images/cats-dogs-val/cats/cat.2082.jpg"
img_data = requests.get(url).content
with open('image_name.jpg', 'wb') as handler:
    handler.write(img_data)


image = Image.open('image_name.jpg')
image = image.resize((32, 32)) 

# Convert the image to a NumPy array
img = np.array(image)

# Transpose and normalize the image data
#img = np.transpose(img, (1, 0, 2)).astype('float32')
image = np.array(img) / 255
imgs = np.expand_dims(image, axis=0)

optim = SGD()
model.compile(loss='categorical_crossentropy', optimizer=optim,
	metrics=['accuracy'])
 
predictions = model.predict(imgs)  # Get prediction probabilities
predicted_class_index = np.argmax(predictions) #Get the index with the highest probability
class_name = classes[predicted_class_index]
print("cette image de chat est prédite comme appartenant à la classe :",predicted_class_index,class_name) # Print predicted class index


plt.imshow(image)
plt.legend(class_name)
plt.show()